<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

app.component


app-routing.module.ts

In [ ]:
import { NgModule } from '@angular/core';
import { RouterModule, Routes } from '@angular/router';
import { AboutComponent } from './about/about.component';
import { MainlogicComponent } from './mainlogic/mainlogic.component';
import { ProductsComponent } from './products/products.component';
import { NewProductComponent } from './new-product/new-product.component';
const routes: Routes = [
  {path:'',component:MainlogicComponent},
  {path:'about',component:AboutComponent},
  {path:'about/:username',component:AboutComponent},
  {path:'products',component:ProductsComponent},
  {path:'newProduct',component:NewProductComponent}
  ];

@NgModule({
  imports: [RouterModule.forRoot(routes)],
  exports: [RouterModule]
})
export class AppRoutingModule { }


app.component.html

In [ ]:
<app-header></app-header>
<router-outlet></router-outlet>


app.component.ts

In [ ]:
import { HttpClientModule } from '@angular/common/http';
import { Component } from '@angular/core';


@Component({
  selector: 'app-root',
//   imports: [MainlogicComponent],
  templateUrl: './app.component.html',
  standalone: false,
  styleUrls: ['./app.component.css']
})
export class AppComponent {
  title = 'form';
}


app.module.ts

In [ ]:
import { NgModule } from '@angular/core';
import { BrowserModule } from '@angular/platform-browser';
import { FormsModule } from '@angular/forms';
import { AppRoutingModule } from './app-routing.module';
import { AppComponent } from './app.component';
import { HeaderComponent } from './header/header.component';
import { MainlogicComponent } from './mainlogic/mainlogic.component';
import { MessagesComponent } from './messages/messages.component';
import { AboutComponent } from './about/about.component';
import { RouterLink} from '@angular/router';
import { HttpClient, HttpClientModule } from '@angular/common/http';
import { ProductsComponent } from './products/products.component';
import { ProductDetailsComponent } from './product-details/product-details.component';
import { NewProductComponent } from './new-product/new-product.component';

@NgModule({
  declarations: [
    AppComponent,
    HeaderComponent,
    MainlogicComponent,
    MessagesComponent,
    AboutComponent,
    ProductsComponent,
    ProductDetailsComponent,
    NewProductComponent
  ],
  imports: [
    BrowserModule,
    AppRoutingModule,
    FormsModule,
    HttpClientModule
  ],
  providers: [HttpClient],
  bootstrap: [AppComponent]
})
export class AppModule { }


services/api

test.service.ts

In [ ]:
import { Injectable } from '@angular/core';

@Injectable({
  providedIn: 'root'
})
export class TestService {
  messages:Array<any>=[];
    constructor() { }

    insertMessage(message: {name:string,email:string,message:string}):void{
        this.messages.push(message);
    }

    getAllMessages():any{
      return this.messages;
    }

    delMessage(index:number):void{
      this.messages.splice(index,1);
    }
}


products/Product.module.ts

In [ ]:
import { HttpClient } from '@angular/common/http';
import { Injectable } from '@angular/core';
import { ProductRepresentation } from './models/productRepresentation';

@Injectable({
  providedIn: 'root'
})
export class ProductService {

  private baseUrl = "https://fakestoreapi.com/";
  constructor(private http:HttpClient) { }

  getAllProductsWithLimit(limit: number = 5){
    const productsUrl:string = `${this.baseUrl}products?limit=${limit}`;
    return this.http.get<ProductRepresentation[]>(productsUrl);
  }

  createProduct(product:ProductRepresentation){
    const productsUrl:string = `${this.baseUrl}products`;
    return this.http.post(productsUrl,product);
  }
}


models/productRepresentation

In [ ]:
import { RatingRepresentation } from "./ratingRepresentation";

export interface ProductRepresentation{
    id?: number,
    title?:string,
    category?:string,
    description?:string,
    image?:string,
    price?:number,
    rating?:RatingRepresentation
}

models/ratingRepresentation

In [ ]:
export interface RatingRepresentation{
    rate?:number,
    count?:number
}

products.component

products.component.html

In [ ]:
<app-product-details
  *ngFor="let product of products"
  [product]="product"
></app-product-details>


products.component.ts

In [ ]:
import { Component, OnInit } from '@angular/core';
import {ProductService} from "../services/api/products/product.service";
import {ProductRepresentation} from "../services/api/products/models/productRepresentation";
@Component({
  selector: 'app-products',
  standalone: false,

  templateUrl: './products.component.html',
  styleUrl: './products.component.css'
})
export class ProductsComponent implements OnInit{
  constructor(private service:ProductService){
  }
  products:ProductRepresentation[]=[];

  ngOnInit(): void {
    this.service.getAllProductsWithLimit()
    .subscribe((data)=>{
      this.products=data;
    })
  }

}


product-details.component

product-details.component.css

In [ ]:
.product-card {
    display: flex;
    border: 1px solid #ccc;
    border-radius: 5px;
    padding: 15px;
    background-color: #fff;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    margin-bottom: 5px;
  }

  .product-image {
    width: 150px;
    height: 150px;
    object-fit: cover;
    border-radius: 5px;
    margin-right: 15px;
  }

  .product-details {
    flex: 1;
  }

  .product-title {
    font-size: 18px;
    margin: 0;
  }

  .product-price {
    font-size: 16px;
    color: #007bff;
  }

  .product-description {
    font-size: 14px;
    line-height: 1.4;
    margin: 5px 0;
    color: #333;
  }

  .product-category {
    font-size: 14px;
    margin-bottom: 5px;
    color: #555;
  }

  .product-rating {
    display: flex;
    align-items: center;
  }

  .rating-value {
    font-size: 16px;
    margin-right: 5px;
    color: #f39c12;
  }

product-details.component.html

In [ ]:
<div class="product-card">
    <img class="product-image" [src]="product.image" [attr.alt]="product.title">
    <div class="product-details">
      <h3 class="product-title"> {{product.title}} </h3>
      <p class="product-price">$  {{product.price}} </p>
      <p class="product-description"> {{product.description}} </p>
      <p class="product-category"> {{product.category}} </p>
      <div class="product-rating">
        <span class="rating-value"> {{product.rating?.rate}} </span>
        <span class="rating-count"> {{ product.rating?.count }}  reviews</span>
      </div>
    </div>
  </div>


product-details.component.ts

In [ ]:
import { Component,Input } from '@angular/core';
import { ProductRepresentation } from '../services/api/products/models/productRepresentation';

@Component({
  selector: 'app-product-details',
  standalone: false,

  templateUrl: './product-details.component.html',
  styleUrl: './product-details.component.css'
})
export class ProductDetailsComponent {
  @Input()
  product: ProductRepresentation = {};
}


new-product.component

new-product.component.css

In [ ]:
form {
    display: flex;
    flex-direction: column;
    max-width: 400px;
    margin: 0 auto;
    padding: 10px;
    background-color: #f8f8f8;
    border: 1px solid #ccc;
    border-radius: 5px;
  }

  div {
    margin-bottom: 15px;
  }

  label {
    font-weight: bold;
  }

  input[type="text"],
  input[type="number"],
  textarea {
    width: 95%;
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
  }

  textarea {
    resize: vertical;
    min-height: 100px;
  }

  button {
    background-color: #007bff;
    color: #fff;
    padding: 10px 20px;
    border: none;
    border-radius: 5px;
    cursor: pointer;
  }

  button:hover {
    background-color: #0056b3;
  }

  /* Error messages */
  .error {
    color: red;
    font-size: 12px;
  }

new-product.component.html

In [ ]:
<h2>Add a New Product</h2>

<form (ngSubmit)="saveProduct()" >
  <div>
    <label for="title">Title:</label>
    <input type="text" id="title" [(ngModel)]="product.title" name="title" required>
  </div>

  <div>
    <label for="price">Price:</label>
    <input type="number" id="price" [(ngModel)]="product.price" name="price" required>
  </div>

  <div>
    <label for="description">Description:</label>
    <textarea id="description"  [(ngModel)]="product.description" name="description" required></textarea>
  </div>

  <div>
    <label for="image">Image URL:</label>
    <input type="text"  [(ngModel)]="product.image" id="image" name="image" required>
  </div>

  <div>
    <label for="category">Category:</label>
    <input type="text"  [(ngModel)]="product.category" id="category" name="category" required>
  </div>

  <button type="submit">Save Product</button>
</form>


new-product.component.ts

In [ ]:
import { Component } from '@angular/core';
import { ProductRepresentation } from '../services/api/products/models/productRepresentation';
import {ProductService} from "../services/api/products/product.service";
import {Router} from "@angular/router";

@Component({
  selector: 'app-new-product',
  standalone: false,

  templateUrl: './new-product.component.html',
  styleUrls: ['./new-product.component.css']
})
export class NewProductComponent {
  product: ProductRepresentation = {};

  constructor(
    private service: ProductService,
    private router: Router
  ) {
  }

  saveProduct() {
    this.service.createProduct(this.product)
      .subscribe({
        next: (result) => {
          this.router.navigate(['products']);
        }
      });
  }
}


messages.component

messages.component.css

In [ ]:


.message-details{
   margin: 10px;
   padding: 5px;
   border: 1px solid #ccc;
   background-color: #f7f7f7;
   border-radius: 4px;
   word-wrap: break-word;
   white-space: pre-wrap;
}

li{
  list-style-type: none;
  word-wrap: break-word;
  white-space: pre-wrap;
}

#button{
  background-color:green;
  margin:10px;
}


messages.component.html

In [ ]:
<div class="message-details">
  <ul>
    <li>Name: {{message.name}}</li>
    <li>Email: {{message.email}}</li>
    <li>Message: {{message.message}}</li>
  </ul>
  <button id="button" (click)="onDelete()">Delete</button>
</div>


messages.component.ts

In [ ]:
import { Component, Input, Output, EventEmitter } from '@angular/core';

@Component({
  selector: 'app-messages',
  standalone: false,
  templateUrl: './messages.component.html',
  styleUrls: ['./messages.component.css']
})
export class MessagesComponent {
  @Input() message:any ='';
    @Input() index:number = -1;

    @Output() del = new EventEmitter();
    onDelete(){
      this.del.emit(this.index);
    }
}


mainlogic.component

mainlogic.component.css

In [ ]:
.form-container {
  max-width: 400px;
  margin: auto;
  padding: 20px;
  border: 1px solid #ccc;
  border-radius: 5px;
  background-color: #f7f7f7;
}

h4{
  text-align: center;
}

.form-ele{
  display:flex;
  padding:1%;
  align-items:center;
}

#Submit{
  background-color:green;
}




mainlogic.component.html

In [ ]:
<div class="form-container">
  <h4 >Sample Form:</h4>
  <form (ngSubmit)="onSubmit()">
    <div class="form-ele">
      <label for="name">Name:</label>
      <input type="text" id="name" required [(ngModel)]="name" name="name">
    </div>
    <div class="form-ele">
      <label for="email">Email:</label>
      <input type="text" id="email" required [(ngModel)]="email" name="email">
    </div>
    <div class="form-ele">
      <label for="message">Message:</label>
      <textarea id="message" rows="4" [(ngModel)]="message" name="message"  required></textarea>
    </div>
    <div class="form-ele">
      <button id="Submit" >Submit</button>
    </div>
  </form>

  <div id="container" *ngIf="toShow">
    <app-messages *ngFor="let ele of messages; index as i"
                  [message]="ele"
                  [index]="i"
                  (del)="deleteMessage($event)">
    </app-messages>
  </div>
</div>



mainlogic.component.ts

In [ ]:
import { Component } from '@angular/core';
import { TestService } from '../services/test.service';


@Component({
  selector: 'app-mainlogic',
  standalone: false,
  templateUrl: './mainlogic.component.html',
  styleUrls: ['./mainlogic.component.css']
})
export class MainlogicComponent {
  name:string='';
      email:string='';
      message:string='';
      toShow:boolean = false;
      messages:Array<any>=[];

      constructor(private ts:TestService){
        this.messages = this.ts.getAllMessages();
        this.toShow = this.messages.length > 0;
      }

      onSubmit(){
        this.toShow = true;
        this.ts.insertMessage({'name':this.name,'email':this.email,'message':this.message});
        this.messages = this.ts.getAllMessages();
      }

      deleteMessage(index: number){
        this.ts.delMessage(index);
        this.messages = this.ts.getAllMessages();
      }
}


header.component

header.component.css

In [ ]:
.menu{
  display:flex;
  justify-content:center;
  background: green;
  color:whilte
}

a{
  padding:2%;
}


header.component.html

In [ ]:
<div class="menu">
  <a routerLink="/" routerLinkActive="active">Home</a>
  <a routerLink="/newProduct" routerLinkActive="active">Add Product</a>
  <a routerLink="/products" routerLinkActive="active">Products</a>
  <a routerLink="/about" routerLinkActive="active">About</a>
</div>


about.component

In [ ]:
about.component.css

In [ ]:
.about-container {
  max-width: 800px;
  margin: 0 auto;
  padding: 20px;
  background-color: #f5f5f5;
  border-radius: 5px;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

h2 {
  color: #333;
}

p {
  color: #555;
  line-height: 1.6;
}


about.component.html

In [ ]:
<div class="about-container">
  <h2>About the Angular Crash Course</h2>
  <p>
    Welcome to the Angular Crash Course! This course is designed for beginners who want to learn Angular,
    one of the most popular JavaScript frameworks for building web applications.
  </p>

  <p>
    In this course, you will learn the fundamentals of Angular, including components, services, routing,
    and HTTP communication. By the end of the course, you will have a solid understanding of how to build
    powerful and interactive web applications using Angular.
  </p>

  <p>
    Whether you are new to web development or have experience with other frameworks, this crash course
    will get you up to speed with Angular in no time.
  </p>
</div>


about.component.ts

In [ ]:
import { Component, OnInit } from '@angular/core';
import { ActivatedRoute } from '@angular/router';
import { ProductService } from '../services/api/products/product.service';
import { ProductRepresentation } from '../services/api/products/models/productRepresentation';

@Component({
  selector: 'app-about',
  standalone: false,

  templateUrl: './about.component.html',
  styleUrl: './about.component.css'
})
export class AboutComponent{
}
